**RESSOURCES HUMAINES - MOURAD**








In [ ]:
#Q1: Performance des représentants commerciaux : Calculer le chiffre d’affaires généré par chaque employé chargé des ventes.

SELECT SUM(od.priceEach*od.quantityOrdered) AS CA, e.lastName AS Nom, e.firstname AS Prénom
FROM customers c
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON od.orderNumber = o.orderNumber
JOIN employees e ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY Nom,Prénom;


In [ ]:
#Q2 : Ratio commandes/paiements par représentant commercial : Identifier les écarts entre les commandes passées et les paiements reçus pour chaque représentant.

WITH CA_C as (
SELECT e.employeeNumber,e.lastName AS Nom, e.firstname AS Prénom,SUM(od.priceEach*od.quantityOrdered) AS CA
FROM customers c
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON od.orderNumber = o.orderNumber
JOIN employees e ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY e.employeeNumber,Nom,Prénom
),
CA_P AS (
SELECT
e.employeeNumber, SUM(p.amount) AS paiement
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN payments p ON c.customerNumber = p.customerNumber
GROUP BY e.employeeNumber
)
SELECT c.Nom,c.Prénom, C.CA,p.paiement, (p.paiement / c.CA) * 100 as ratio
FROM CA_C C
JOIN CA_P p ON c.employeeNumber = p.employeeNumber
ORDER BY ratio DESC


In [ ]:
#Q3 : Performance des bureaux : Mesurer le chiffre d’affaires généré par chaque bureau.

SELECT e.officeCode AS Office,SUM(od.priceEach*od.quantityOrdered) AS CA
FROM orderdetails od
JOIN orders o ON od.orderNumber = o.orderNumber
JOIN customers c ON o.customerNumber = c.customerNumber
JOIN employees e ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY Office
ORDER BY CA DESC;


**VENTES - JOHANE**

In [ ]:
# Q1 :  Chiffre d’affaires par mois et par région + taux d’évolution mensuel :Suivre les revenus générés par région et par mois pour identifier les tendances géographiques.

CREATE VIEW q1_1 as (
SELECT Mois,Continents, CA, Annee,
(CA - LAG(CA,1) OVER (PARTITION BY Continents ORDER BY mois)) / LAG(CA,1) OVER (PARTITION BY Continents ORDER BY mois) * 100 AS Taux
FROM (
SELECT DATE_FORMAT(o.Orderdate, "%m") as Mois, SUM(od.priceEach*od.quantityOrdered) AS CA, ofi.territory as Continents,
DATE_FORMAT(o.Orderdate, "%y") as Annee
FROM orderdetails od
JOIN orders o ON od.orderNumber = o.orderNumber
JOIN customers c ON c.customerNumber = o.customerNumber
JOIN employees e ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN offices ofi ON  ofi.officeCode = e.officeCode
GROUP BY DATE_FORMAT(o.Orderdate, "%m"), ofi.territory, Annee
) AS eee
ORDER BY mois,Continents)
;

In [ ]:
# J'ai traité les 2 questions dans la meme requete, il y a quand meme deux requetes pour separer les 5 + vendus et les 5 - vendus.

# Q2 : Produits les plus/moins vendus par catégorie : Identifier les produits les plus performants dans chaque catégorie.
# Q3 : La marge brute par produit et par catégorie : Mesurer la marge brute et en déduire les produits/catégories les plus/moins rentables.

CREATE VIEW q_2_max as (

select categorie, productName,quantite, vente, cout, marge, taux_marge
from (
select p.productName, p.productLine as categorie, SUM(od.quantityOrdered) as quantite,
rank() over (partition by p.productLine order by  SUM(od.quantityOrdered) desc) as ranking,
SUM(od.quantityOrdered*od.priceEach) as vente,
SUM(p.buyPrice*od.quantityOrdered) as cout,
SUM(od.quantityOrdered*od.priceEach)-SUM(p.buyPrice*od.quantityOrdered) as marge,
ROUND((SUM(od.quantityOrdered*od.priceEach)-SUM(p.buyPrice*od.quantityOrdered)) / (SUM(od.quantityOrdered*od.priceEach)) * 100 ) as taux_marge
from products p
join orderdetails od on p.productCode = od.productCode
group by p.productLine, p.productName
order by  categorie
) as e
where ranking < 6
);

CREATE VIEW q_2_min as (

select categorie, productName, quantite, vente, cout, marge, taux_marge
from (
select p.productName, p.productLine as categorie, SUM(od.quantityOrdered) as quantite,
rank() over (partition by p.productLine order by  SUM(od.quantityOrdered) asc) as ranking,
SUM(od.quantityOrdered*od.priceEach) as vente,
SUM(p.buyPrice*od.quantityOrdered) as cout,
SUM(od.quantityOrdered*od.priceEach)-SUM(p.buyPrice*od.quantityOrdered) as marge,
ROUND((SUM(od.quantityOrdered*od.priceEach)-SUM(p.buyPrice*od.quantityOrdered)) / (SUM(od.quantityOrdered*od.priceEach)) * 100 ) as taux_marge
from products p
join orderdetails od on p.productCode = od.productCode
group by p.productLine, p.productName
order by  categorie
) as e
where ranking < 6
);


In [ ]:
# Q4 : Taux d’évolution mensuel des ventes par catégorie : Mesurer la performance de chaque catégorie de produit dans le temps.
CREATE VIEW q4_ as (
select Mois, CA, productLine, Annee,
((CA-LAG(CA,1) OVER (PARTITION by productLine order by Mois))/LAG(CA,1) OVER(PARTITION by productLine order by Mois))*100 As taux
from(
select DATE_FORMAT(o.orderDate, "%m") as Mois, DATE_FORMAT(o.orderDate, "%y") as Annee, p.productLine, SUM(od.priceEach * od.quantityOrdered) as CA
FROM orderdetails od
join orders o on od.orderNumber = o.orderNumber
join products p on od.productCode=p.productCode
group by productLine, Mois, Annee
) as E
order by productLine, Annee, Mois
)

In [ ]:
# Q5 : Panier moyen : Mesurer le panier moyen des commandes.
SELECT SUM(od.priceEach * od.quantityOrdered) / COUNT(DISTINCT od.orderNumber) AS PanierMoyen
FROM orderdetails od;

In [ ]:
# Q6 : Taux de retour des clients (repeat customers) : Mesurer la fidélité des clients en identifiant ceux qui passent plusieurs commandes.
#calcul du taux global de retour :
CREATE VIEW q6 as (
select count(customer)/(select count(*) from customers)*100 as taux
from(
select c.customerName as customer, count(o.customerNumber) as nb_commandes
from customers c
join orders o on c.customerNumber = o.customerNumber
group by customer, c.customerNumber
having count(o.orderNumber) > 1
order by nb_commandes desc
) as a );

#calcul du top 3 des clients avec le plus de commande (les plus fideles)
CREATE VIEW q6_top as (
select c.customerName as customer, count(o.customerNumber) as nb_commandes
from customers c
join orders o on c.customerNumber = o.customerNumber
group by customer, c.customerNumber
having count(o.orderNumber) > 1
order by nb_commandes desc
limit 3)

**FINANCES - PAUL**

In [ ]:
# Q1 : Clients générant le plus/moins de revenus : Identifier les clients générant le plus de revenus pour mieux les fidéliser.

In [ ]:
SELECT c.customerName, SUM(p.amount) as total
FROM customers c
JOIN payments p
  ON c.customerNumber = p.customerNumber
GROUP BY c.customerName
ORDER BY total DESC;

In [ ]:
# Q2 : Taux de recouvrement des créances par client : Identifier les clients ayant un montant élevé de commandes non payées.

In [ ]:
WITH ca_client AS (
  SELECT o.customerNumber, SUM(od.quantityOrdered * od.priceEach) AS total_commande
  FROM orders o
  JOIN orderdetails od ON o.orderNumber = od.orderNumber
  GROUP BY o.customerNumber
),
payer_client AS (
  SELECT customerNumber, SUM(amount) AS total_payer
  FROM payments
  GROUP BY customerNumber
)
SELECT
  c.customerName,
  ca.total_commande,
  p.total_payer,
  (ca.total_commande - p.total_payer) AS montant_impaye
FROM customers c
JOIN ca_client ca ON c.customerNumber = ca.customerNumber
LEFT JOIN payer_client p ON c.customerNumber = p.customerNumber
WHERE ca.total_commande > 0
ORDER BY montant_impaye DESC;

In [ ]:
# Q3 : Croissance des ventes par trimestre : Identifier les tendances saisonnières ou les opportunités de croissance.

In [ ]:
SELECT
YEAR(o.orderDate) AS annee,
MONTH(o.orderDate) AS num_mois,
MONTHNAME(o.orderDate) AS mois,
SUM(od.quantityOrdered * od.priceEach) AS ca
FROM orders o
JOIN orderdetails od
ON o.orderNumber = od.orderNumber
GROUP BY YEAR(o.orderDate), MONTH(o.orderDate), MONTHNAME(o.orderDate)
ORDER BY annee, ca DESC;

In [ ]:
# Q4 : Montant moyen des paiements + clients en dessous de la moyenne : Évaluer la capacité de paiement des clients.

In [ ]:
WITH moyenne AS (
SELECT
p.customerNumber,
c.customerName,
ROUND(AVG(p.amount), 2) AS moyenne_paiement
FROM payments p
JOIN customers c
ON c.customerNumber = p.customerNumber
GROUP BY p.customerNumber, c.customerName
),
moyenne_calc AS (
SELECT
*,
AVG(moyenne_paiement) OVER () AS moyenne_globale
FROM moyenne
)
SELECT *
FROM moyenne_calc
WHERE moyenne_paiement < moyenne_globale
ORDER BY moyenne_paiement DESC;

In [ ]:
# Q5 : Taux de paiement par délai : Identifier combien de temps les clients mettent pour payer après une commande.

**LOGISTIQUE - EDDY**

In [ ]:
# Q1 : Stock des produits sous seuil critique : Identifier les produits dont le stock est faible pour éviter les ruptures.
#Identifier les produits :
SELECT productCode, productName, quantityInStock
FROM products
#Quantifier et classer :
WHERE quantityInStock < (
        SELECT AVG(quantityInStock)
        FROM products )
ORDER BY quantityInStock DESC;

In [ ]:
# Q2 : Durée moyenne de traitement des commandes + commandes au-dessus de la moyenne de livraison : Mesurer l’efficacité opérationnelle en analysant le temps entre la date de commande et la date d’expédition.
#Comparer la durée moyenne :
SELECT
    orderNumber,
    orderDate,
    shippedDate,
    DATEDIFF(shippedDate, orderDate) AS duree_traitement
FROM orders
WHERE shippedDate IS NOT NULL;

#Commandes plus lentes que la moyenne :
SELECT
    orderNumber,
    orderDate,
    shippedDate,
    DATEDIFF(shippedDate, orderDate) AS duree_traitement
FROM orders
WHERE shippedDate IS NOT NULL
  AND DATEDIFF(shippedDate, orderDate) >
      (
        SELECT AVG(DATEDIFF(shippedDate, orderDate))
        FROM orders
        WHERE shippedDate IS NOT NULL
      );


In [ ]:
# Q3 : Taux d’écoulement des stocks : Mesurer la rapidité avec laquelle les stocks sont vendus pour chaque produit.
#Taux d'écoulements des stocks :

#  ------------   UPDATE  --------------

CREATE OR REPLACE VIEW vue_duree_traitement_produit AS
WITH moyenne_traitement AS (
  SELECT AVG(DATEDIFF(shippedDate, orderDate)) AS duree_moyenne
  FROM orders
)
SELECT
  o.orderNumber,
  o.orderDate,
  o.shippedDate,
  DATEDIFF(o.shippedDate, o.orderDate) AS delai_traitement,
  mt.duree_moyenne,
  p.productCode,
  p.productName
FROM
  orders o
  JOIN moyenne_traitement mt
  JOIN orderdetails od ON o.orderNumber = od.orderNumber
  JOIN products p ON od.productCode = p.productCode
WHERE
  DATEDIFF(o.shippedDate, o.orderDate) > mt.duree_moyenne
ORDER BY delai_traitement DESC;


In [ ]:
# Q4 : Taux de commandes livrées en retard : Identifier les problèmes logistiques et améliorer les délais de livraison.

#   -----------   UPDATE    ------------

CREATE OR REPLACE VIEW vue_taux_ecoulement_produit AS
SELECT
    o.productCode,
    p.productName,
    SUM(o.quantityOrdered) AS total_vendu,
    p.quantityInStock,
    ROUND(SUM(o.quantityOrdered) / p.quantityInStock, 2) AS taux_ecoulement_p
FROM orderdetails o
JOIN products p
  ON o.productCode = p.productCode
GROUP BY
    o.productCode,
    p.productName,
    p.quantityInStock
ORDER BY
    taux_ecoulement_p DESC;

#    ------------- UPDATE -------------

CREATE OR REPLACE VIEW vue_commandes_hors_delais AS
SELECT
  o.orderNumber                        AS numero_commande,
  o.orderDate                          AS date_commande,
  YEAR(o.orderDate)                    AS annee_commande,
  MONTH(o.orderDate)                   AS mois_commande,
  o.requiredDate                       AS date_prevue_livraison,
  o.shippedDate                        AS date_commande_livree,
  DATEDIFF(o.shippedDate, o.requiredDate) AS retard_jrs,
  CASE
    WHEN o.shippedDate > o.requiredDate THEN 'En retard'
    ELSE 'Dans les temps'
  END AS statut_livraison,
  ROUND(
    (SELECT COUNT(*) FROM orders WHERE shippedDate > requiredDate) /
    (SELECT COUNT(*) FROM orders) * 100, 2
  ) AS taux_retard_global
FROM orders o
WHERE o.shippedDate IS NOT NULL AND o.requiredDate IS NOT NULL;


